In [ ]:
path = "../datasets/bitly_usagov/example.txt"

In [ ]:
import json
with open(path) as f:
    records = [json.loads(line) for line in f]

In [ ]:
time_zones = [rec["tz"] for rec in records]

In [ ]:
time_zones = [rec["tz"] for rec in records if "tz" in rec]
time_zones[:10]

In [ ]:
def get_counts(sequence):
    counts = {}
    for x in sequence:
        if x in counts:
            counts[x] += 1
        else:
            counts[x] = 1
    return counts

In [ ]:
from collections import defaultdict

def get_counts2(sequence):
    counts = defaultdict(int) # values will initialize to 0
    for x in sequence:
        counts[x] += 1
    return counts

In [ ]:
counts = get_counts(time_zones)
counts["America/New_York"]
len(time_zones)

In [ ]:
def top_counts(count_dict, n=10):
    value_key_pairs = [(count, tz) for tz, count in count_dict.items()]
    value_key_pairs.sort()
    return value_key_pairs[-n:]

In [ ]:
top_counts(counts)

In [ ]:
from collections import Counter

counts = Counter(time_zones)
counts.most_common(10)

In [ ]:
frame = pd.DataFrame(records)

In [ ]:
frame.info()
frame["tz"].head()

In [ ]:
tz_counts = frame["tz"].value_counts()
tz_counts.head()

In [ ]:
clean_tz = frame["tz"].fillna("Missing")
clean_tz[clean_tz == ""] = "Unknown"
tz_counts = clean_tz.value_counts()
tz_counts.head()

In [ ]:
plt.figure(figsize=(10, 4))

In [ ]:
import seaborn as sns
subset = tz_counts.head()
#! figure,id=usa_gov_counts,title="Top time zones in the 1.usa.gov sample data"
sns.barplot(y=subset.index, x=subset.to_numpy())

In [ ]:
frame["a"][1]
frame["a"][50]
frame["a"][51][:50]  # long line

In [ ]:
results = pd.Series([x.split()[0] for x in frame["a"].dropna()])
results.head(5)
results.value_counts().head(8)

In [ ]:
cframe = frame[frame["a"].notna()].copy()

In [ ]:
cframe["os"] = np.where(cframe["a"].str.contains("Windows"),
                        "Windows", "Not Windows")
cframe["os"].head(5)

In [ ]:
by_tz_os = cframe.groupby(["tz", "os"])

In [ ]:
agg_counts = by_tz_os.size().unstack().fillna(0)
agg_counts.head()

In [ ]:
indexer = agg_counts.sum("columns").argsort()
indexer.values[:10]

In [ ]:
count_subset = agg_counts.take(indexer[-10:])
count_subset

In [ ]:
agg_counts.sum(axis="columns").nlargest(10)

In [ ]:
plt.figure()

In [ ]:
count_subset = count_subset.stack()
count_subset.name = "total"
count_subset = count_subset.reset_index()
count_subset.head(10)
sns.barplot(x="total", y="tz", hue="os",  data=count_subset)

In [ ]:
def norm_total(group):
    group["normed_total"] = group["total"] / group["total"].sum()
    return group

results = count_subset.groupby("tz").apply(norm_total)

In [ ]:
plt.figure()

In [ ]:
sns.barplot(x="normed_total", y="tz", hue="os",  data=results)

In [ ]:
g = count_subset.groupby("tz")
results2 = count_subset["total"] / g["total"].transform("sum")